In [34]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [35]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "cpi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "30"

    if len(sheet_names) == 11:
        return "Sheet2"

    raise RuntimeError(sheet_names)

In [36]:
en_to_vn = {
    "CONSUMER PRICE INDEXES": "CHỈ SỐ GIÁ TIÊU DÙNG",
    "Food and catering services": "Hàng ăn và dịch vụ ăn uống",
    "Grain food": "Lương thực",
    "Foodstuff": "Thực phẩm",
    "Outdoor eating and drinking": "Ăn uống ngoài gia đình",
    "Drinks and tobacco": "Đồ uống và thuốc lá",
    "Textile, footgear and hats": "May mặc, mũ nón và giày dép ",
    "Housing and construction materials": "Nhà ở và vật liệu xây dựng",
    "Family appliances and tools": "Thiết bị và đồ dùng gia đình",
    "Medicaments and health service": "Thuốc và dịch vụ y tế",
    "Medical service": "Dịch vụ y tế",
    "Transport": "Giao thông",
    "Postal and communicational service": "Bưu chính viễn thông",
    "Education": "Giáo dục",
    "Educational service": "Dịch vụ giáo dục",
    "Culture, entertainment and tourism": "Văn hoá, giải trí và du lịch",
    "Others": "Hàng hóa và dịch vụ khác",
    "GOLD PRICE INDEXES": "CHỈ SỐ GIÁ VÀNG",
    "US DOLLAR PRICE INDEXES": "CHỈ SỐ GIÁ ĐÔ LA MỸ",
    "CORE INFLATION": "LẠM PHÁT CƠ BẢN",
}

In [44]:
def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Hàng ăn và dịch vụ ăn uống")
                | (df[df.columns[1]] == "Food and catering services")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Lương thực")
                | (df[df.columns[1]] == "Grain food")
                | (df[df.columns[1]] == " Lương thực")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if len(df.columns) == 7:
        df = df.drop(
            columns=[df.columns[1], df.columns[3], df.columns[5]]
        )
        df.columns = ["Hàng hóa", "M-YOY", "M-MOM", "YTD-YOY"]
    elif len(df.columns) == 6 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[4]])
        df.columns = ["Hàng hóa", "M-YOY", "M-MOM", "YTD-YOY"]
    elif len(df.columns) == 6:
        df = df.drop(columns=[df.columns[1], df.columns[3]])
        df.columns = ["Hàng hóa", "M-YOY", "M-MOM", "YTD-YOY"]
    elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
        df = df.drop(columns=[df.columns[1]])
        df["A"] = df[df.columns[1]]
        df.columns = ["Hàng hóa", "M-YOY", "M-MOM", "YTD-YOY"]
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1]])
        df.columns = ["Hàng hóa", "M-YOY", "M-MOM", "YTD-YOY"]
    else:
        raise RuntimeError(df.columns)

    assert len(df.columns) == 4, list(df.columns)

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "CHỈ SỐ GIÁ TIÊU DÙNG")
                    | ((df[df.columns[0]] == "CONSUMER PRICE INDEXES"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "CONSUMER PRICE INDEXES"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    df = df[pd.to_numeric(df["M-YOY"], errors="coerce").notnull()]
    df = df[pd.to_numeric(df["YTD-YOY"], errors="coerce").notnull()]

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


xl = pd.ExcelFile("datas/2017-03-29-Bieu-3-2017-1.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_cpi(xl, sheet_name, "2017-03-29")

,Hàng hóa,M-YOY,MOM,YTD-YOY,Date
7,CHỈ SỐ GIÁ TIÊU DÙNG,104.645396,100.208,104.961552,2017-03-29
9,Hàng ăn và dịch vụ ăn uống,100.106906,99.1291,100.986121,2017-03-29
10,Lương thực,102.075976,100.1563,102.265152,2017-03-29
11,Thực phẩm,99.12843,98.7754,100.400453,2017-03-29
12,Ăn uống ngoài gia đình,101.611167,99.5107,101.829086,2017-03-29
13,Đồ uống và thuốc lá,101.667901,99.9758,101.707909,2017-03-29
14,"May mặc, mũ nón và giày dép",101.049762,99.8812,101.186123,2017-03-29
15,Nhà ở và vật liệu xây dựng,105.300223,100.4966,104.534412,2017-03-29
16,Thiết bị và đồ dùng gia đình,100.981384,100.0583,100.985408,2017-03-29
17,Thuốc và dịch vụ y tế,135.929031,107.5085,149.693252,2017-03-29


In [38]:
import os
import contextlib

dfs_cpi_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        xl = pd.ExcelFile(file_path)

        df_cpi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_cpi_origin.append(df_cpi)
    except Exception as e:
        print(file_name)
        raise e

In [39]:
dfs_cpi = pd.concat(dfs_cpi_origin, ignore_index=True, axis=0)
dfs_cpi["Date"] = pd.to_datetime(dfs_cpi["Date"], dayfirst=False)
dfs_cpi

,Hàng hóa,M-YOY,MOM,YTD-YOY,Date
0,CHỈ SỐ GIÁ TIÊU DÙNG,103.976409,101.0373,103.673209,2024-02-29
1,Hàng ăn và dịch vụ ăn uống,104.234976,101.7146,103.266519,2024-02-29
2,Lương thực,117.355578,101.7492,116.490991,2024-02-29
3,Thực phẩm,102.137883,101.9847,100.891369,2024-02-29
4,Ăn uống ngoài gia đình,104.203352,101.0358,103.903106,2024-02-29
...,...,...,...,...,...
2420,Dịch vụ giáo dục,112.45,100,112.45,2014-01-29
2421,"Văn hoá, giải trí và du lịch",102.9,100.21,102.9,2014-01-29
2422,Đồ dùng và dịch vụ khác,104.91,100.63,104.91,2014-01-29
2423,CHỈ SỐ GIÁ VÀNG,75.57,98.18,75.57,2014-01-29


In [40]:
dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "M-YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "M-YOY"].astype(float) - 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "YTD-YOY"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "YTD-YOY"].astype(float) - 100
)
dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "M-MOM"] = (
    dfs_cpi.loc[dfs_cpi["Hàng hóa"] != "LẠM PHÁT CƠ BẢN", "M-MOM"].astype(float) - 100
)
dfs_cpi

,Hàng hóa,M-YOY,MOM,YTD-YOY,Date
0,CHỈ SỐ GIÁ TIÊU DÙNG,3.976409,1.0373,3.673209,2024-02-29
1,Hàng ăn và dịch vụ ăn uống,4.234976,1.7146,3.266519,2024-02-29
2,Lương thực,17.355578,1.7492,16.490991,2024-02-29
3,Thực phẩm,2.137883,1.9847,0.891369,2024-02-29
4,Ăn uống ngoài gia đình,4.203352,1.0358,3.903106,2024-02-29
...,...,...,...,...,...
2420,Dịch vụ giáo dục,12.45,0.0,12.45,2014-01-29
2421,"Văn hoá, giải trí và du lịch",2.9,0.21,2.9,2014-01-29
2422,Đồ dùng và dịch vụ khác,4.91,0.63,4.91,2014-01-29
2423,CHỈ SỐ GIÁ VÀNG,-24.43,-1.82,-24.43,2014-01-29


In [47]:
dfs_cpi.to_csv('csv/cpi.csv')

In [48]:
dfs_cpi = pd.read_csv('csv/cpi.csv')

In [49]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "CHỈ SỐ GIÁ TIÊU DÙNG")
        | (dfs_cpi["Hàng hóa"] == "LẠM PHÁT CƠ BẢN")
    )
    & (dfs_cpi["Date"] > "2015-01-01")
]

fig = px.area(df_show, x="Date", y="M-YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [50]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "CHỈ SỐ GIÁ TIÊU DÙNG")
        | (dfs_cpi["Hàng hóa"] == "LẠM PHÁT CƠ BẢN")
    )
    & (dfs_cpi["Date"] > "2020-01-01")
]

fig = px.area(df_show, x="Date", y="M-MOM", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [57]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_cpi[
    (
        (dfs_cpi["Hàng hóa"] == "Lương thực")
        | (dfs_cpi["Hàng hóa"] == "Nhà ở và vật liệu xây dựng")
        | (dfs_cpi["Hàng hóa"] == "Giao thông")
        | (dfs_cpi["Hàng hóa"] == "Giáo dục")
        | (dfs_cpi["Hàng hóa"] == "Thuốc và dịch vụ y tế")
    )
    & (dfs_cpi["Date"] > "2020-01-01")
]

fig = px.area(df_show, x="Date", y="M-YOY", color="Hàng hóa")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()